#### Install `psyopg2`

In [1]:
!pip install psycopg2

The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |████████████████████████████████| 2.7MB 664kB/s 


#### Import dependencies

In [113]:
import os, urllib, psycopg2
import pandas as pd
import numpy as np

#### Setup environment

In [104]:
os.environ["ELEPHANT_URL"] = # Enter the url on this page https://api.elephantsql.com/console/

In [105]:
urllib.parse.uses_netloc.append("postgres")
url = urllib.parse.urlparse(os.environ["ELEPHANT_URL"])

#### Load data

In [115]:
titanic = pd.read_csv('titanic.csv')
renamed = ['survived', 'pclass', 'name', 'sex', 'age', 'siblings_or_spouses_aboard', 'parents_or_children_aboard', 'fare']
titanic = titanic.rename(columns={k: v for k,v in zip(titanic.columns.values, sql_columns)})
titanic.survived = titanic.survived.astype(np.bool)

#### Connect to database

In [147]:
psycho = psycopg2.connect(database=url.path[1:],
    user=url.username,
    password=url.password,
    host=url.hostname,
    port=url.port
)

In [148]:
cursor = psycho.cursor()

#### Create table

In [149]:
cursor.execute("drop table if exists titanic; drop type if exists sex;")

In [150]:
cursor.execute(
    """
    create type sex as enum ('male', 'female');
    
    create table titanic(
        survived boolean,
        pclass smallint,
        name text,
        sex sex,
        age real,
        siblings_or_spouses_aboard smallint,
        parents_or_children_aboard smallint,
        fare real
    )
    """
)

#### Insert data

In [ ]:
titanic.apply(
    lambda row: 
        cursor.execute(
            """
            insert into titanic(
                survived, 
                pclass, 
                name, 
                sex, 
                age, 
                siblings_or_spouses_aboard, 
                parents_or_children_aboard, 
                fare
            ) values (%s, %s, %s, %s, %s, %s, %s, %s)
            """, 
            row.values
        ), 
    axis=1
);

#### Verify

In [156]:
cursor.execute("select * from titanic")
rows = cursor.fetchall()

In [157]:
len(rows), titanic.shape

(887, (887, 8))

In [158]:
rows[0]

(False, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)

In [159]:
titanic[:1]

,survived,pclass,name,sex,age,siblings_or_spouses_aboard,parents_or_children_aboard,fare
0,False,3,Mr. Owen Harris Braund,male,22.0,1,0,7.25
